# Transformation

### Import libraries. 

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("delta")\
    .option('inferSchema', 'true')\
        .option('header', 'true')\
            .load("abfss://bronze@dlnetflix.dfs.core.windows.net/netflix_titles")

display(df.limit(5))

duration_minutes duration_seasons type title date_added release_year rating description show_id _rescued_data 90 null Movie Norm of the North: King Sized Adventure 9/9/2019 2019 TV-PG Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first. 81145628 null 94 null Movie Jandino: Whatever it Takes 9/9/2016 2016 TV-MA "Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of ""Sex on Fire"" in his comedy show." 80117401 null null 1 TV Show Transformers Prime 9/8/2018 2013 TV-Y7-FV With the help of three human allies, the Autobots once again protect Earth from the onslaught of the Decepticons and their leader, Megatron. 70234439 null null 1 TV Show Transformers: Robots in Disguise 9/8/2018 2016 TV-Y7 When a prison ship crash unleashes hundreds of Decepticons on Earth, Bumblebee leads a new Autobot force to protect humankind. 80058654 null 99 null Movie #realityhigh 9/8/2017 2017 TV-14 When nerdy high schooler Dani finally attracts the interest of her longtime crush, she lands in the cross hairs of his ex, a social media celebrity. 80125979 null

In [0]:
df = df.fillna({'duration_minutes': 0, 'duration_seasons': 1})
display(df.limit(5))

duration_minutes duration_seasons type title date_added release_year rating description show_id _rescued_data 90 1 Movie Norm of the North: King Sized Adventure 9/9/2019 2019 TV-PG Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first. 81145628 null 94 1 Movie Jandino: Whatever it Takes 9/9/2016 2016 TV-MA "Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of ""Sex on Fire"" in his comedy show." 80117401 null 0 1 TV Show Transformers Prime 9/8/2018 2013 TV-Y7-FV With the help of three human allies, the Autobots once again protect Earth from the onslaught of the Decepticons and their leader, Megatron. 70234439 null 0 1 TV Show Transformers: Robots in Disguise 9/8/2018 2016 TV-Y7 When a prison ship crash unleashes hundreds of Decepticons on Earth, Bumblebee leads a new Autobot force to protect humankind. 80058654 null 99 1 Movie #realityhigh 9/8/2017 2017 TV-14 When nerdy high schooler Dani finally attracts the interest of her longtime crush, she lands in the cross hairs of his ex, a social media celebrity. 80125979 null

In [0]:
df= df.withColumn('duration_minutes', df['duration_minutes'].cast(IntegerType()))\
     .withColumn('duration_seasons', df['duration_seasons'].cast(IntegerType()))

df.printSchema()

root
 |-- duration_minutes: integer (nullable = true)
 |-- duration_seasons: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- description: string (nullable = true)
 |-- show_id: string (nullable = true)
 |-- _rescued_data: string (nullable = true)



In [0]:
df = df.withColumn('rating',split(df['rating'],'-')[0])\
    .drop('_rescued_data')\
        .withColumn('title', split(df['title'],':')[0])


display(df.limit(5))

duration_minutes duration_seasons type title date_added release_year rating description show_id 90 1 Movie Norm of the North 9/9/2019 2019 TV Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first. 81145628 94 1 Movie Jandino 9/9/2016 2016 TV "Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of ""Sex on Fire"" in his comedy show." 80117401 0 1 TV Show Transformers Prime 9/8/2018 2013 TV With the help of three human allies, the Autobots once again protect Earth from the onslaught of the Decepticons and their leader, Megatron. 70234439 0 1 TV Show Transformers 9/8/2018 2016 TV When a prison ship crash unleashes hundreds of Decepticons on Earth, Bumblebee leads a new Autobot force to protect humankind. 80058654 99 1 Movie #realityhigh 9/8/2017 2017 TV When nerdy high schooler Dani finally attracts the interest of her longtime crush, she lands in the cross hairs of his ex, a social media celebrity. 80125979

In [0]:
df= df.withColumn('type_flag', when(df['type']=='TV Show', 1)\
    .when(df['type']=='Movie', 2).otherwise(0))

display(df.limit(5))

duration_minutes duration_seasons type title date_added release_year rating description show_id type_flag 90 1 Movie Norm of the North 9/9/2019 2019 TV Before planning an awesome wedding for his grandfather, a polar bear king must take back a stolen artifact from an evil archaeologist first. 81145628 2 94 1 Movie Jandino 9/9/2016 2016 TV "Jandino Asporaat riffs on the challenges of raising kids and serenades the audience with a rousing rendition of ""Sex on Fire"" in his comedy show." 80117401 2 0 1 TV Show Transformers Prime 9/8/2018 2013 TV With the help of three human allies, the Autobots once again protect Earth from the onslaught of the Decepticons and their leader, Megatron. 70234439 1 0 1 TV Show Transformers 9/8/2018 2016 TV When a prison ship crash unleashes hundreds of Decepticons on Earth, Bumblebee leads a new Autobot force to protect humankind. 80058654 1 99 1 Movie #realityhigh 9/8/2017 2017 TV When nerdy high schooler Dani finally attracts the interest of her longtime crush, she lands in the cross hairs of his ex, a social media celebrity. 80125979 2

In [0]:
df = df.withColumn('duration_rank', dense_rank().over(Window.orderBy(desc('duration_minutes'))))

display(df.limit(5))

duration_minutes duration_seasons type title date_added release_year rating description show_id type_flag duration_rank 312 1 Movie Black Mirror 12/28/2018 2018 TV In 1984, a young programmer begins to question reality as he adapts a dark fantasy novel into a video game. A mind-bending tale with multiple endings. 80988062 2 1 228 1 Movie Sangam 12/31/2019 1964 TV Returning home from war after being assumed dead, a pilot weds the woman he has long loved, unaware that she had been planning to marry his best friend. 60002818 2 2 224 1 Movie Lagaan 12/8/2017 2001 PG In 1890s India, an arrogant British commander challenges the harshly taxed residents of Champaner to a high-stakes cricket match. 60020906 2 3 214 1 Movie Jodhaa Akbar 10/1/2018 2008 TV In 16th-century India, what begins as a strategic alliance between a Mughal emperor and a Hindu princess becomes a genuine opportunity for true love. 70090035 2 4 209 1 Movie The Irishman 11/27/2019 2019 R Hit man Frank Sheeran looks back at the secrets he kept as a loyal member of the Bufalino crime family in this acclaimed film from Martin Scorsese. 80175798 2 5

In [0]:
df.groupBy('type').count().show()

+-------+-----+
|   type|count|
+-------+-----+
|   NULL|    1|
|TV Show| 1969|
|  Movie| 4265|
|   1944|    1|
+-------+-----+



In [0]:
df.write.format("delta")\
    .mode("overwrite")\
        .option("path", "abfss://silver@dlnetflix.dfs.core.windows.net/netflix_titles")\
            .save()